<a href="https://colab.research.google.com/github/ParthikB/Neural-Style-Transfer/blob/master/neural_style_transfer_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torchvision.models as models

In [0]:
# Defining the Feature Layers we need respectively

# styleLayers = ['block1_conv2', 
#                'block2_conv2', 
#                'block3_conv3', 
#                'block4_conv3', 
#                'block5_conv3']

styleLayers = [2, 7, 14, 23, 32]

# contentLayer = ['block3_conv2']

contentLayer = [12]

numContentLayers = len(contentLayer) # Number of Content Layers
numStyleLayers   = len(styleLayers)  # Number of Style Layers

## Getting the model

In [0]:
# Defining Function to import model (VGG19)
def getModel():

  # Loading Model from tf
  model = models.vgg19(pretrained=True) #----------------------------are parameters freezed?

  # Features of the Respective Layers
  # contentFeatures = [model.get_layer(name).output for name in contentLayer]
  # styleFeatures   = [model.get_layer(name).output for name in styleLayers]
  
  contentFeatures = [list(vgg19.children())[0][layerNumber] for layerNumber in contentLayer] #----------------------- where are the outputs?
  styleFeatures   = [list(vgg19.children())[0][layerNumber] for layerNumber in styleLayers]

  modelOutput = contentFeatures + styleFeatures

  return models.Model(model.input, modelOutput) #---------------------has to reframe this

In [0]:
# Defining GRAM MATRIX
def gram(x):

  # number of channels
  channels = int(x.shape[-1])
  
  # reshaping to channel first
  a = x.view(-1, channels)
  n = a.shape[0]
  
  # gram matrix
  gram = torch.matmul(a, a, transpose_a=True)
  
  return gram / tf.cast(n, tf.float32) #-------------------reframe this tf.cast


# Defining CONTENT COST
def contentCost(contentFeatures, generateFeatures):
  return torch.mean(torch.stack((contentFeatures-generateFeatures)**2)) #---------------------- is the mean correct?


# Defining STYLE COST
def styleCost(styleFeatures, generateFeatures):
  styleGram = gram(styleFeatures)
  return torch.mean(torch.stack((styleGram-generateFeatures)**2)) #---------------------- is the mean correct?

In [0]:
"""  
  Helper function to compute our content and style feature representations.
 
  This function will simply load and preprocess both the content and style 
  images from their path. Then it will feed them through the network to obtain
  the outputs of the intermediate layers. 
  
  Arguments:
    model: The model that we are using.
    content_path: The path to the content image.
    style_path: The path to the style image
    
  Returns:
    returns the style features and the content features. 
"""

def loadImage(path_to_img):
  max_dim = 512
  img2show = Image.open(path_to_img)
  long = max(img2show.size)
  scale = max_dim/long
  img = img2show.resize((round(img2show.size[0]*scale), round(img2show.size[1]*scale)), Image.ANTIALIAS)
  
  img = image.img_to_array(img)
  
  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img2show, axis=0)
  return img2show, img
 

def urlToImage(url):
  resp = urllib.request.urlopen(url)
  img = np.asarray(bytearray(resp.read()), dtype='uint8')
  img = cv2.imdecode(img, cv2.IMREAD_COLOR)
  img2show = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  img = np.expand_dims(img2show, axis=0)
  return img2show, img


def inputImageAndPreprocess(path):
  # Loading the image and reshaping it according to VGG19 requirements.
  if path[:5]=='https':
    print("Loading Image from Internet...")
    img2show, img = urlToImage(path)
  else:
    print("Loading Image from Local...")
    img2show, img = loadImage(path)
  
  # Preprocessing the img according to VGG19 requirements
  img = tf.keras.applications.vgg19.preprocess_input(img) #-------------instead of this, just use the below normalization while feeding the image to the network! 
  # normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

  return img2show, img


def getFeatures(contentPath, stylePath, model):
  
  # Loading and pre-processing the images
  _, content   = inputImageAndPreprocess(contentPath)
  _, style     = inputImageAndPreprocess(stylePath)

  # Defining the respective outputs from our model
  contentOutputs = model(content) #----------------------what is this?
  styleOutputs   = model(style)
  
  # Extracting out the different features from the model output
  contentFeatures = [contentFeature[0] for contentFeature in contentOutputs[numStyleLayers:]]
  styleFeatures = [styleFeature[0] for styleFeature in styleOutputs[:numStyleLayers]]

  return contentFeatures, styleFeatures


# Deprocessing Image to save locally
def deprocessImage(processed_img):
  x = processed_img.copy()
  if len(x.shape) == 4:
    x = np.squeeze(x, 0)
  assert len(x.shape) == 3, ("Input to deprocess image must be an image of "
                             "dimension [1, height, width, channel] or [height, width, channel]")
  if len(x.shape) != 3:
    raise ValueError("Invalid input to deprocessing image")
  
  # perform the inverse of the preprocessiing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68
  x = x[:, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')
  return x

In [3]:
vgg19 = models.vgg19(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:23<00:00, 24.9MB/s]


In [33]:
list(vgg19.children())

[Sequential(
   (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU(inplace=True)
   (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU(inplace=True)
   (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (6): ReLU(inplace=True)
   (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (8): ReLU(inplace=True)
   (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
   (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (11): ReLU(inplace=True)
   (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (13): ReLU(inplace=True)
   (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (15): ReLU(inplace=True)
   (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (17): ReLU(inplace=T